In [58]:
import pandas as pd
from tqdm import tqdm
from itables import show as table_show

In [59]:
ENTREPOT_PATH = '~/Bureau/utils/data/'
METEO_PATH = '~/Bureau/utils/data/meteo/'
SPATIAL_PATH = './data/spatial/'

In [60]:
# obtention des donnnées de la CAN :
directory = '~/Bureau/Exports/20240513/non_anonyme/'
file = 'AGS_20240513_exports_agronomes_interventions_synthetisees.csv'
df_can = pd.read_csv(directory+file, sep='@')

In [71]:
df_studied = df_can.loc[df_can['intervention_id'].isin(studied_ids)]

In [72]:
table_show(pd.melt(df_studied[['intervention_id', 'especes_semees', 'densite_semis', 'unite_semis', 'traitement_chimique_semis', 'inoculation_biologique_semis']], id_vars=['intervention_id']))

In [63]:
print(list(df_studied['intervention_id'].values))

['fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_64a496f9-3d67-4532-847a-0d38a1f5c794']


In [64]:
donnees = {}

def import_df(df_name, path_data, sep, index_col=None):
    donnees[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep, index_col=index_col, low_memory=False)

def import_dfs(df_names, path_data, sep = ',', index_col=None, verbose=False):
    for df_name in tqdm(df_names) : 
        if(verbose) :
            print(" - ", df_name)
        import_df(df_name, path_data, sep, index_col=index_col)

tables_entrepot = [
    'intervention_synthetise', 'semence', 'composant_culture', 'espece', 'utilisation_intrant_synthetise', 'action_synthetise', 'composant_action_semis',
    'noeuds_synthetise', 'connection_synthetise', 'noeuds_synthetise_restructure', 'connection_synthetise_restructure'
]

tables_performance = [
]

# import des données de l'entrepôt avec la colonne 'id' en index 
import_dfs(tables_entrepot, ENTREPOT_PATH, sep = ',',index_col='id', verbose=False)

100%|██████████| 11/11 [00:14<00:00,  1.36s/it]


In [65]:
studied_ids = [
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_b29be8ca-b760-46ea-9d63-2a78ddbd8233', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_f6a5eb34-66be-4c6a-a005-fba158c058b3', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_ef2316bd-b80d-45aa-921b-21af6d56ecba', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_5e3f0abb-4a03-4da0-a26c-d73bdecfcb74', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_cd159e5c-e76f-4efb-9380-b53dccd5444c', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_a24ca460-d863-40f9-bab6-24817300384e', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_64a496f9-3d67-4532-847a-0d38a1f5c794', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_7ad44155-633e-4521-bfcb-7bf482128e3c', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_2efe5e73-195f-4b2b-9078-3b66202a8396', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_c2f8dad3-845a-4185-bf71-a35384930a43'
]

In [66]:
left = donnees['noeuds_synthetise']
right = donnees['noeuds_synthetise_restructure']
donnees['noeuds_synthetise'] = pd.merge(left, right, left_index=True, right_index=True, how='left')


left = donnees['connection_synthetise']
right = donnees['connection_synthetise_restructure']
donnees['connection_synthetise'] = pd.merge(left, right, left_index=True, right_index=True, how='left')

In [67]:
donnees['intervention_synthetise_test'] = donnees['intervention_synthetise'].loc[
    donnees['intervention_synthetise'].index.isin(studied_ids)
]

donnees['action_synthetise_test'] = donnees['action_synthetise'].loc[
    donnees['action_synthetise']['intervention_synthetise_id'].isin(donnees['intervention_synthetise_test'].index)
]
donnees['connection_synthetise_test'] = donnees['connection_synthetise'].loc[
    (donnees['connection_synthetise'].index.isin(list(donnees['intervention_synthetise_test']['connection_synthetise_id'])))
]
donnees['composant_action_semis_test'] = donnees['composant_action_semis'].loc[
    donnees['composant_action_semis']['action_id'].isin(donnees['action_synthetise_test'].index)
]

donnees['noeuds_synthetise_test'] = donnees['noeuds_synthetise'].loc[
    (donnees['noeuds_synthetise'].index.isin(list(donnees['connection_synthetise_test']['source_noeuds_synthetise_id']))) |
    (donnees['noeuds_synthetise'].index.isin(list(donnees['connection_synthetise_test']['cible_noeuds_synthetise_id'])))
]

donnees['composant_culture_test'] = donnees['composant_culture'].loc[
    donnees['composant_culture']['culture_id'].isin(donnees['noeuds_synthetise_test']['culture_id']) |
    donnees['composant_culture']['culture_id'].isin(donnees['connection_synthetise_test']['culture_intermediaire_id'])
]

donnees['espece_test'] = donnees['espece'].loc[
    donnees['espece'].index.isin(donnees['composant_culture_test']['espece_id'])
]
donnees['semence_test']= donnees['semence'].loc[
    donnees['semence']['espece_id'].isin(donnees['composant_culture_test'].index)
]

donnees['utilisation_intrant_synthetise_test'] = donnees['utilisation_intrant_synthetise'].loc[
    donnees['utilisation_intrant_synthetise']['intervention_synthetise_id'].isin(donnees['intervention_synthetise_test'].index)
]

In [68]:

path='./'
donnees['semence_test'].to_csv(path+'semence'+'.csv')
donnees['composant_culture_test'].to_csv(path+'composant_culture'+'.csv')
donnees['espece_test'].to_csv(path+'espece'+'.csv')
donnees['utilisation_intrant_synthetise_test'].to_csv(path+'utilisation_intrant_synthetise'+'.csv')

In [69]:
donnees['intervention_synthetise_test'].loc['fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_64a496f9-3d67-4532-847a-0d38a1f5c794']

nom                                                                  Semis CIPAN et céréales
type                                                                                   SEMIS
commentaire                                                                              NaN
rang                                                                                       1
date_debut                                                                             20/08
date_fin                                                                               20/08
freq_spatiale                                                                            1.0
freq_temporelle                                                                          1.0
psci_intervention                                                                        1.0
debit_de_chantier                                                                        1.8
debit_de_chantier_unite                                               